<a href="https://colab.research.google.com/github/RMhanovernorwichschools/neural_network_prac/blob/master/saving_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

#bringing in clothes images again just so we do create a model which can be saved
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000] #taking the first 1000
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0 #reshape each image to 1D array where pixel values are listed in a line instead of a
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0   #28 by 28 grid. the -1 means that the resulting array will have a row for each example
                                                                #since it functions like a placeholder.
#the "/ 255.0" is b/c values from 0 to 1 are better for the program.

In [27]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model
#the model uses some dropout to counter overfitting and has only one hidden layer of 512 nodes.


# Create a basic model instance
model = create_model()
model.summary()
#check to make sure it is in desired format

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Callbacks Saving
### An important NOTE about checkpoints:

Using Colab, you need to save the *string under which the save is filed*, not the variable name.

In [28]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 896/1000 [=========================>....] - ETA: 0s - loss: 1.1942 - acc: 0.6518
Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 1s 722us/sample - loss: 1.1467 - acc: 0.6650 - val_loss: 0.6777 - val_acc: 0.7920
Epoch 2/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.4184 - acc: 0.8750
Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 313us/sample - loss: 0.4196 - acc: 0.8770 - val_loss: 0.5625 - val_acc: 0.8250
Epoch 3/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.2864 - acc: 0.9173
Epoch 00003: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 313us/sample - loss: 0.2858 - acc: 0.9180 - val_loss: 0.475

In [29]:
#since the checkpoint was saved with a name, it can be "reloaded" at any point. This means the weights (which are the saved part) can be applied
#to any model with the same architecture
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))
#this can be demonstrated by applying to this currently-untrained model

1000/1000 [==============================] - 0s 140us/sample - loss: 2.3891 - acc: 0.0870
Untrained model, accuracy:  8.70%


In [30]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 66us/sample - loss: 0.4013 - acc: 0.8720
Restored model, accuracy: 87.20%


That way you can always use the checkpoint's name to load the weights, and use callbacks so training always saves to the checkpoint name.

In [31]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"    #loads a different kind of checkpoint (specific string for specific function)
checkpoint_dir = os.path.dirname(checkpoint_path)     #same format as used with training_1 checkpoint above

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)   #specifically, "period=n" means saves every n epochs

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

! ls {checkpoint_dir}


Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00005: saving model to training_2/cp-0005.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00010: saving model to training_2/cp-0010.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00015: saving model to training_2/cp-0015.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00020: saving model to training_2/cp-0020.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00025: saving model to training_2/cp-0025.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00030: saving model to training_2/cp-0030.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00035: saving model to training_2/cp-0035.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00040: saving model to training_2/cp-0040.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00045: saving model to training_2/cp-0045.ckpt

Consider 

In [32]:
print(tf.train.latest_checkpoint(checkpoint_dir)) #this is what one would load for the saved weights after the most recent save (epoch 50)
model = create_model()
model.load_weights("training_2/cp-0050.ckpt")
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))
#proof that it works

training_2/cp-0050.ckpt
1000/1000 [==============================] - 0s 171us/sample - loss: 0.4932 - acc: 0.8750
Restored model, accuracy: 87.50%


## Manual Saving

In [33]:
#manual saving is also possible (not using callbacks)
#the format for this is shown below

# Save the weights
model.save_weights('name./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('name./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))
#'name' can be replaced by anything with the proper character types (including nothing)


Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 168us/sample - loss: 0.4932 - acc: 0.8750
Restored model, accuracy: 87.50%


## Saving the Entire Model

In [0]:
# Save entire model to a HDF5 file; format = "model.save('[name]'.h5)"
model.save('my_model.h5')

#NOTE: will need to redo "model.configure" for training

In [36]:
#Loading an entire model; format = "keras.model.load_model('[name]'.h5)"

# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
1000/1000 [==============================] - 0s 171us/sample - loss: 0.4932 - acc: 0.8750
Restored model, accuracy: 87.50%


In [37]:
#can also save models specifically as saved models. This will be less useful as Colab doesn't save variables outside of a session
#first, create a "saved model" with the format; "[name] = tf.contrib.saved_model.save_keras_model([model], './saved_models')"
saved_model_path = tf.contrib.saved_model.save_keras_model(model, "./saved_models")
!ls saved_models/ #see it in the directory


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.


Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_models/1555774665/saved_model.pb
1555774665


In [41]:
#how to load
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.summary()

# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
1000/1000 [==============================] - 0s 176us/sample - loss: 0.4932 - acc: 0.8750
Restored model, accuracy: 87.50%


In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.